In [66]:
%pip install pgmpy
%pip install pyitlib

Note: you may need to restart the kernel to use updated packages.Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple



You should consider upgrading via the 'C:\Users\rukam\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simpleNote: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\rukam\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [67]:
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import math_ops, array_ops
from tensorflow.keras.constraints import Constraint
from tensorflow.keras.activations import softmax
class softmax_weight(Constraint):
    """Constrains weight tensors to be under softmax `."""
  
    def __init__(self,feature_uniques):
        idxs = math_ops.cumsum([0] + feature_uniques)
        idxs = [i.numpy() for i in idxs]
        self.feature_idxs = [
            (idxs[i],idxs[i+1]) for i in range(len(idxs)-1)
        ]
  
    def __call__(self, w):     
        w_new = [
            math_ops.log(softmax(w[i:j,:], axis=0))
            for i,j in self.feature_idxs
        ]
        return tf.concat(w_new, 0)
  
    def get_config(self):
        return {'feature_idxs': self.feature_idxs}

In [68]:
def sample(*arrays, size=None, frac=None):
    '''
    random sample from arrays.
    
    Note: arrays must be equal-length
    
    size = None (default) indicate that return a permutation of given arrays.
    '''
    if len(arrays) < 1:
        return None
    if frac is not None and frac <= 1 and frac > 0:
        size = int(len(arrays[0]) * frac)
    if size is None:
        size = len(arrays[0])
        
    random_idxs = np.random.permutation(len(arrays[0]))[:size]
    results = []
    for arr in arrays:
        results.append(arr[random_idxs])
    return results

# GANBLR

In [75]:
def elr_loss(KL_LOSS):
  def loss(y_true, y_pred):
    return tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)+ KL_LOSS
  return loss

In [76]:
#from tensorflow.keras.layers import Concatenate
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder
class GANBLR:
    def __init__(self, log=True):
        self.g = None
        self.d = None
        self.log = log
        self.g_history = []
        self.d_history = []
        
        self.batch_size = 32 # default value
        self.feature_uniques = []
        self.class_unique = 0
        self.g_input_dim = 0
        self.d_input_dim = 0
        self.y_counts = []
    
    def generate(self, size=None, ohe=False):
        from pgmpy.models import NaiveBayes
        from pgmpy.sampling import BayesianModelSampling
        from pgmpy.factors.discrete import TabularCPD
        #basic varibles
        weights = self.g.get_weights()[0]
        n_features = len(self.feature_uniques)
        n_classes = weights.shape[1]
        n_samples = np.sum(self.class_counts)
        #cut weights by feature uniques
        idxs = np.cumsum([0] + self.feature_uniques)
        feature_idxs  = [(idxs[i],idxs[i+1]) for i in range(len(idxs)-1)]
        feature_names = [str(i) for i in range(n_features)]
        #get cpd of features
        feature_probs = np.exp(weights)
        feature_cpd_probs = [feature_probs[start:end,:] for start, end in feature_idxs]
        feature_cpd_probs = [p/p.sum(axis=0,keepdims=1) for p in feature_cpd_probs]
        feature_cpds = [
            TabularCPD(name, n_unique, table, evidence=['y'], evidence_card=[n_classes])
            for name, n_unique, table in zip(feature_names, self.feature_uniques, feature_cpd_probs)
        ]
        #get cpd of label
        y_probs = (self.class_counts/n_samples).reshape(-1,1)
        y_cpd = TabularCPD('y', n_classes, y_probs)
    
        #define the model
        elr = NaiveBayes(feature_names, 'y')
        elr.add_cpds(y_cpd, *feature_cpds)
        #sampling
        sample_size = n_samples if size is None else size
        result = BayesianModelSampling(elr).forward_sample(size=sample_size)
        sorted_result = result[feature_names + ['y']].values
        #return
        syn_X, syn_y = sorted_result[:,:-1], sorted_result[:,-1]
    
        if ohe:
            ohe_syn_X = [np.eye(b)[syn_X[:,i]] for i, b in enumerate(self.feature_uniques)]
            ohe_syn_X = np.hstack(ohe_syn_X)
            return ohe_syn_X, syn_y
        else:
            return sorted_result
        
    def fit(self, X, y, epochs
                , batch_size=32, warm_up_epochs=10, categories_=None):
        
        if categories_ is None:
            categories_ = 'auto'
        ohe = OneHotEncoder(categories=categories_).fit(X)
        ohe_X = ohe.transform(X).toarray()
        #feature_uniques = [len(np.unique(X[:,i])) for i in range(X.shape[1])]
        self.feature_uniques = [len(c) for c in ohe.categories_]
        y_unique, y_counts = np.unique(y, return_counts=True)
        self.class_unique = len(y_unique)
        self.class_counts = y_counts
        self.g_input_dim = np.sum(self.feature_uniques)
        self.d_input_dim = X.shape[1]
        self.batch_size = batch_size
        self._build_g()
        self._build_d()
        
        #warm up
        self.g.fit(ohe_X, y, epochs=warm_up_epochs, batch_size=batch_size)
        syn_data  = self.generate(size=len(X))
        #real_data = np.concatenate([X, y.reshape(-1,1)], axis=-1)
        for i in range(epochs):
            #prepare data
            real_label = np.ones(len(X))
            syn_label = np.zeros(len(X))
            disc_label = np.concatenate([real_label, syn_label])    
            disc_X = np.vstack([X, syn_data[:,:-1]])    
            disc_X, disc_label = sample(disc_X, disc_label, frac=0.8)
            #train d
            self._train_d(disc_X, disc_label)
            prob_fake = self.d.predict(X)
            ls = np.mean(-np.log(np.subtract(1,prob_fake)))
            #train g
            self._train_g(ohe_X, y, loss=ls)
            syn_data = self.generate(size=len(X))

    def _train_g(self, X, y, epochs=1, loss=None):
        if loss is not None:
            clear_session()
            self._build_g(weights=self.g.get_weights(), loss=loss)
            self._build_d(weights=self.d.get_weights())
            
        history = self.g.fit(X, y, epochs=epochs, batch_size=self.batch_size)
        if self.log:
            self.g_history.append(history.history)
    
    def _train_d(self, X, y, epochs=1):
        history = self.d.fit(X, y, batch_size=self.batch_size, epochs=epochs)      
        if self.log:
            self.d_history.append(history.history)  
            
    def _build_g(self, weights=None, loss=None):
        if loss is None:
            loss = elr_loss(0)
        else:
            loss = elr_loss(loss)
        constraint = softmax_weight(self.feature_uniques)
        g = tf.keras.Sequential()    
        g.add(Dense(self.class_unique, input_dim=self.g_input_dim, activation='softmax',kernel_constraint=constraint))
        g.compile(loss=loss, optimizer='adam', metrics=['accuracy'])
        self.g = g
        
        if weights is not None:
            g.set_weights(weights)
        return g
    
    def _build_d(self, weights=None):
        d = tf.keras.Sequential()
        d.add(Dense(1, input_dim=self.d_input_dim, activation='sigmoid'))
        d.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        self.d = d
        
        if weights is not None:
            d.set_weights(weights)
        return d

# Ganblr Extend

In [77]:
#from tensorflow.keras.layers import Concatenate
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder
class GANBLR_Ext:
    def __init__(self, log=True):
        self.g = None
        self.d = None
        self.log = log
        self.g_history = []
        self.d_history = []
        
        self.batch_size = 32 # default value
        self.feature_uniques = []
        self.class_unique = 0
        self.g_input_dim = 0
        self.d_input_dim = 0
        self.y_counts = []
    
    def generate(self, size=None, ohe=False):
        from pgmpy.models import NaiveBayes
        from pgmpy.sampling import BayesianModelSampling
        from pgmpy.factors.discrete import TabularCPD
        #basic varibles
        weights = self.g.get_weights()[0]
        n_features = len(self.feature_uniques)
        n_classes = weights.shape[1]
        n_samples = np.sum(self.class_counts)
        #cut weights by feature uniques
        idxs = np.cumsum([0] + self.feature_uniques)
        feature_idxs  = [(idxs[i],idxs[i+1]) for i in range(len(idxs)-1)]
        feature_names = [str(i) for i in range(n_features)]
        #get cpd of features
        feature_probs = np.exp(weights)
        feature_cpd_probs = [feature_probs[start:end,:] for start, end in feature_idxs]
        feature_cpd_probs = [p/p.sum(axis=0,keepdims=1) for p in feature_cpd_probs]
        feature_cpds = [
            TabularCPD(name, n_unique, table, evidence=['y'], evidence_card=[n_classes])
            for name, n_unique, table in zip(feature_names, self.feature_uniques, feature_cpd_probs)
        ]
        #get cpd of label
        y_probs = (self.class_counts/n_samples).reshape(-1,1)
        y_cpd = TabularCPD('y', n_classes, y_probs)
    
        #define the model
        elr = NaiveBayes(feature_names, 'y')
        elr.add_cpds(y_cpd, *feature_cpds)
        #sampling
        sample_size = n_samples if size is None else size
        result = BayesianModelSampling(elr).forward_sample(size=sample_size)
        sorted_result = result[feature_names + ['y']].values
        #return
        syn_X, syn_y = sorted_result[:,:-1], sorted_result[:,-1]
    
        if ohe:
            ohe_syn_X = [np.eye(b)[syn_X[:,i]] for i, b in enumerate(self.feature_uniques)]
            ohe_syn_X = np.hstack(ohe_syn_X)
            return ohe_syn_X, syn_y
        else:
            return sorted_result
        
    def fit(self, X, y, epochs
                , batch_size=32, warm_up_epochs=10, categories_=None):
        
        if categories_ is None:
            categories_ = 'auto'
        ohe = OneHotEncoder(categories=categories_).fit(X)
        ohe_X = ohe.transform(X).toarray()
        #feature_uniques = [len(np.unique(X[:,i])) for i in range(X.shape[1])]
        self.feature_uniques = [len(c) for c in ohe.categories_]
        y_unique, y_counts = np.unique(y, return_counts=True)
        self.class_unique = len(y_unique)
        self.class_counts = y_counts
        self.g_input_dim = np.sum(self.feature_uniques)
        self.d_input_dim = X.shape[1] + self.class_unique
        self.batch_size = batch_size
        self._build_g()
        self._build_d()
        
        real_label = np.ones(len(X))
        syn_label = np.zeros(len(X))
        #warm up
        self.g.fit(ohe_X, y, epochs=warm_up_epochs, batch_size=batch_size)
        syn_data  = self.generate(size=len(X))
        #real_data = np.concatenate([X, y.reshape(-1,1)], axis=-1)
        for i in range(epochs):
            #prepare data 
            disc_label = np.concatenate([real_label, syn_label])
            disc_X = np.vstack([X, syn_data[:,:-1]])
            g_logits = self.g(tf.convert_to_tensor(ohe.transform(disc_X).toarray())).numpy()
            disc_X = np.hstack([g_logits, disc_X])
            disc_X, disc_label = sample(disc_X, disc_label, frac=0.8)
            #train d
            self._train_d(disc_X, disc_label)
            prob_fake = self.d.predict(np.hstack([g_logits[:len(X)], X]))
            ls = np.mean(-np.log(np.subtract(1,prob_fake)))
            #train g
            self._train_g(ohe_X, y, loss=ls)
            syn_data = self.generate(size=len(X))

    def _train_g(self, X, y, epochs=1, loss=None):
        if loss is not None:
            g_weights = self.g.get_weights()
            d_weights = self.d.get_weights()
            clear_session()
            self._build_g(weights=g_weights, loss=loss)
            self._build_d(weights=d_weights)
            
        history = self.g.fit(X, y, epochs=epochs, batch_size=self.batch_size)
        if self.log:
            self.g_history.append(history.history)
    
    def _train_d(self, X, y, epochs=1):
        history = self.d.fit(X, y, batch_size=self.batch_size, epochs=epochs)      
        if self.log:
            self.d_history.append(history.history)  
            
    def _build_g(self, weights=None, loss=None):
        if loss is None:
            loss = elr_loss(0)
        else:
            loss = elr_loss(loss)
        constraint = softmax_weight(self.feature_uniques)
        g = tf.keras.Sequential()    
        g.add(Dense(self.class_unique, input_dim=self.g_input_dim, activation='softmax',kernel_constraint=constraint))
        g.compile(loss=loss, optimizer='adam', metrics=['accuracy'])
        self.g = g
        
        if weights is not None:
            g.set_weights(weights)
        return g
    
    def _build_d(self, weights=None):
        d = tf.keras.Sequential()
        d.add(Dense(1, input_dim=self.d_input_dim, activation='sigmoid'))
        d.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        self.d = d
        
        if weights is not None:
            d.set_weights(weights)
        return d

# Test

In [96]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split

df = pd.read_csv('car_value.csv',dtype='category')
df1 = OrdinalEncoder().fit_transform(df).astype('int')
print(df.shape)
X = df1[:,0:-1]
y = df1[:,-1]  
print(len(np.unique(y)))

(12960, 9)
5


In [97]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
categories = OneHotEncoder().fit(X).categories_

In [98]:
epochs = 50
batch_size = 32

clear_session()
ganblr_ext = GANBLR_Ext()
ganblr_ext.fit(X_train, y_train, epochs=epochs, warm_up_epochs=5,categories_=categories)
ganblr = GANBLR()
ganblr.fit(X_train, y_train, epochs=epochs, warm_up_epochs=5,categories_=categories)

Epoch 1/5
203/203 [==============================] - 0s 1ms/step - loss: 1.4299 - accuracy: 0.3100
Epoch 2/5
203/203 [==============================] - 0s 1ms/step - loss: 1.0296 - accuracy: 0.6759
Epoch 3/5
203/203 [==============================] - 0s 1ms/step - loss: 0.8309 - accuracy: 0.8576
Epoch 4/5
203/203 [==============================] - 0s 1ms/step - loss: 0.7016 - accuracy: 0.8901
Epoch 5/5
203/203 [==============================] - 0s 1ms/step - loss: 0.6102 - accuracy: 0.9039


Generating for node: 6: 100%|██████████| 9/9 [00:00<00:00, 34.48it/s]


203/203 [==============================] - 0s 1ms/step - loss: 1.2579 - accuracy: 0.9079


Generating for node: 6: 100%|██████████| 9/9 [00:00<00:00, 35.86it/s]


203/203 [==============================] - 0s 2ms/step - loss: 1.1829 - accuracy: 0.9113


...

Generating for node: 6: 100%|██████████| 9/9 [00:00<00:00, 37.50it/s]


203/203 [==============================] - 0s 2ms/step - loss: 0.9343 - accuracy: 0.9278


Generating for node: 6: 100%|██████████| 9/9 [00:00<00:00, 26.87it/s]


203/203 [==============================] - 0s 2ms/step - loss: 0.9035 - accuracy: 0.9264


Generating for node: 6: 100%|██████████| 9/9 [00:00<00:00, 25.64it/s]


In [99]:
# g_loss_log = [d['loss'] for d in ganblr.g_history]
# d_loss_log = [d['loss'] for d in ganblr.d_history]
# 
# plt.plot(g_loss_log, label='generator loss')
# plt.plot(d_loss_log, label='discriminator loss')
# plt.legend()

In [100]:
# g_loss_log = [d['loss'] for d in ganblr_ext.g_history]
# d_loss_log = [d['loss'] for d in ganblr_ext.d_history]
# 
# plt.plot(g_loss_log, label='generator loss')
# plt.plot(d_loss_log, label='discriminator loss')
# plt.legend()

In [101]:
ganblr_syn_data = ganblr.generate(len(X_train), ohe=True)
ganblr_ext_syn_data = ganblr_ext.generate(len(X_train), ohe=True)

Generating for node: 6: 100%|██████████| 9/9 [00:00<00:00, 28.21it/s]


In [102]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [103]:
ohe = OneHotEncoder(categories=categories, sparse=False)
lr2 = LogisticRegression().fit(*ganblr_ext_syn_data)
lr3 = LogisticRegression().fit(ohe.fit_transform(X_train), y_train)
lr1 = LogisticRegression().fit(*ganblr_syn_data)

C:\Users\rukam\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\rukam\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

In [106]:
pred1 = lr1.predict(ohe.fit_transform(X_test))
pred2 = lr2.predict(ohe.fit_transform(X_test))
pred3 = lr3.predict(ohe.fit_transform(X_test))

In [107]:
print('TSTR on GANBLR:\t', accuracy_score(y_test, pred1))
print('TSTR on GANBLR Extend:\t', accuracy_score(y_test, pred2))
print('TRTR:\t', accuracy_score(y_test, pred3))

TSTR on GANBLR:	 0.8447530864197531
TSTR on GANBLR Extend:	 0.8356481481481481
TRTR:	 0.9246913580246914


### result backup

adult:
- TSTR on GANBLR:	 0.7512796363785267
- TSTR on GANBLR Extend:	 0.8077883788542648
- TRTR:	 0.8730191228860407

car

- TSTR on GANBLR:	 0.8148148148148148
- TSTR on GANBLR Extend:	 0.8090277777777778
- TRTR:	 0.90625

nursery

- TSTR on GANBLR:	 0.8447530864197531
- TSTR on GANBLR Extend:	 0.8356481481481481
- TRTR:	 0.9246913580246914